# Capitulo 1: Estructura Basica de Prompts

**Nota:** Los prompts de ejemplo se mantienen en ingles ya que las tecnicas de ingenieria de prompts son independientes del idioma y Claude responde de manera mas predecible en ingles.

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu clave API y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## Leccion

Anthropic ofrece dos APIs, la antigua [API de Text Completions](https://docs.anthropic.com/claude/reference/complete_post) y la actual [API de Messages](https://docs.anthropic.com/claude/reference/messages_post). Para este tutorial, usaremos exclusivamente la API de Messages.

Como minimo, una llamada a Claude usando la API de Messages requiere los siguientes parametros:
- `model`: el [nombre del modelo en la API](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations) del modelo que deseas utilizar

- `max_tokens`: el numero maximo de tokens a generar antes de detenerse. Ten en cuenta que Claude puede detenerse antes de alcanzar este maximo. Este parametro solo especifica el numero maximo absoluto de tokens a generar. Ademas, es una parada *forzada*, lo que significa que puede hacer que Claude deje de generar a mitad de una palabra o una oracion.

- `messages`: un arreglo de mensajes de entrada. Nuestros modelos estan entrenados para operar en turnos conversacionales alternados entre `user` y `assistant`. Al crear un nuevo `Message`, especificas los turnos conversacionales previos con el parametro messages, y el modelo genera el siguiente `Message` en la conversacion.
  - Cada mensaje de entrada debe ser un objeto con `role` y `content`. Puedes especificar un unico mensaje con rol `user`, o puedes incluir multiples mensajes de `user` y `assistant` (deben alternarse en ese caso). El primer mensaje siempre debe usar el rol `user`.

Tambien hay parametros opcionales, como:
- `system`: el prompt de sistema - mas sobre esto a continuacion.
  
- `temperature`: el grado de variabilidad en la respuesta de Claude. Para estas lecciones y ejercicios, hemos establecido `temperature` en 0.

Para una lista completa de todos los parametros de la API, visita nuestra [documentacion de la API](https://docs.anthropic.com/claude/reference/messages_post).

### Ejemplos

Veamos como responde Claude a algunos prompts correctamente formateados. Para cada una de las siguientes celdas, ejecuta la celda (`shift+enter`), y la respuesta de Claude aparecera debajo del bloque.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

Ahora veamos algunos prompts que no incluyen el formato correcto de la API de Messages. Para estos prompts mal formateados, la API de Messages devuelve un error.

Primero, tenemos un ejemplo de una llamada a la API de Messages que carece de los campos `role` y `content` en el arreglo `messages`.

In [ ]:
# Obtener la respuesta de Claude
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Imprimir la respuesta de Claude
print(response[0].text)

Aqui hay un prompt que no alterna entre los roles `user` y `assistant`.

In [ ]:
# Obtener la respuesta de Claude
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Imprimir la respuesta de Claude
print(response[0].text)

Los mensajes de `user` y `assistant` **DEBEN alternarse**, y los mensajes **DEBEN comenzar con un turno de `user`**. Puedes tener multiples pares de `user` y `assistant` en un prompt (como si simularas una conversacion de multiples turnos). Tambien puedes poner palabras en un mensaje final de `assistant` para que Claude continue desde donde lo dejaste (mas sobre esto en capitulos posteriores).

#### Prompts de Sistema

Tambien puedes usar **prompts de sistema**. Un prompt de sistema es una forma de **proporcionar contexto, instrucciones y directrices a Claude** antes de presentarle una pregunta o tarea en el turno de "User".

Estructuralmente, los prompts de sistema existen separados de la lista de mensajes de `user` y `assistant`, y por lo tanto pertenecen a un parametro `system` separado (observa la estructura de la funcion auxiliar `get_completion` en la seccion de [Configuracion](#configuracion) del notebook).

Dentro de este tutorial, donde podriamos utilizar un prompt de sistema, te hemos proporcionado un campo `system` en tu funcion de completions. Si no deseas usar un prompt de sistema, simplemente establece la variable `SYSTEM_PROMPT` como una cadena vacia.

#### Ejemplo de Prompt de Sistema

In [ ]:
# Prompt de sistema
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT, SYSTEM_PROMPT))

Por que usar un prompt de sistema? Un **prompt de sistema bien escrito puede mejorar el rendimiento de Claude** de diversas maneras, como aumentar la capacidad de Claude para seguir reglas e instrucciones. Para mas informacion, visita nuestra documentacion sobre [como usar prompts de sistema](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/system-prompts) con Claude.

Ahora nos sumergiremos en algunos ejercicios. Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del notebook de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 1.1 - Contar hasta Tres](#ejercicio-11---contar-hasta-tres)
- [Ejercicio 1.2 - Prompt de Sistema](#ejercicio-12---prompt-de-sistema)

### Ejercicio 1.1 - Contar hasta Tres
Usando el formato adecuado de `user` / `assistant`, edita el `PROMPT` a continuacion para que Claude **cuente hasta tres.** La salida tambien indicara si tu solucion es correcta.

In [ ]:
# Prompt - este es el unico campo que debes cambiar
PROMPT = "[Replace this text]"

# Obtener la respuesta de Claude
response = get_completion(PROMPT)

# Funcion para calificar la correccion del ejercicio
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Imprimir la respuesta de Claude y la calificacion correspondiente
print(response)
print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Ejercicio 1.2 - Prompt de Sistema

Modifica el `SYSTEM_PROMPT` para que Claude responda como si fuera un nino de 3 anos.

In [ ]:
# Prompt de sistema - este es el unico campo que debes cambiar
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Obtener la respuesta de Claude
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Funcion para calificar la correccion del ejercicio
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Imprimir la respuesta de Claude y la calificacion correspondiente
print(response)
print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Felicidades!

Si has resuelto todos los ejercicios hasta este punto, estas listo para pasar al siguiente capitulo. Feliz creacion de prompts!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompts mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT))

In [ ]:
# Obtener la respuesta de Claude
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Imprimir la respuesta de Claude
print(response[0].text)

In [ ]:
# Obtener la respuesta de Claude
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Imprimir la respuesta de Claude
print(response[0].text)

In [ ]:
# Prompt de sistema
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Imprimir la respuesta de Claude
print(get_completion(PROMPT, SYSTEM_PROMPT))